In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
def read_df(df_name):
    return pd.read_csv(df_name)

steam_spy_df = read_df("steam_spy_final.csv")
metacritics_df = read_df("metacritic.csv")

In [3]:
steam_spy_df.drop(columns=["Unnamed: 0"], inplace=True)
metacritics_df.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
final_df = pd.concat([steam_spy_df], axis=1)

In [5]:
final_df.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [6]:
final_df.rename(columns={'average_forever' : 'average_playtime'}, inplace=True)
final_df.rename(columns={'median_forever' : 'mid_playtime'}, inplace=True) #hetzion
final_df

,name,developer,publisher,positive,negative,owners,average_playtime,mid_playtime,price
0,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5718191,761211,"50,000,000 .. 100,000,000",28846,6493,0
1,Dota 2,Valve,Valve,1467658,297030,"100,000,000 .. 200,000,000",38219,964,0
2,Grand Theft Auto V,Rockstar North,Rockstar Games,1153983,208800,"20,000,000 .. 50,000,000",12460,6441,1480
3,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1146769,892200,"50,000,000 .. 100,000,000",21155,6620,0
4,Terraria,Re-Logic,Re-Logic,951689,20646,"20,000,000 .. 50,000,000",6126,1739,499
...,...,...,...,...,...,...,...,...,...
9634,HEDE Game Engine,Hede Games,Hede Games,0,7,"50,000 .. 100,000",0,0,99
9635,League Space,EURO GAMES STUDIO,EURO GAMES STUDIO (ESP),0,1,"50,000 .. 100,000",0,0,1199
9636,RagDoll MadDoll,Team Booky,Dystopian Edge Publishing,0,1,"50,000 .. 100,000",0,0,124
9637,Operation: VICUS,"Ilja Soutchilin, Tom Berger","Ilja Soutchilin, Tom Berger",0,2,"50,000 .. 100,000",0,0,799


In [7]:
final_df.to_csv("final_df.csv")


In [8]:
metacritics_df.sort_values(by=['name', 'score'], ascending=[True, False], ignore_index=True, inplace=True)
metacritics_df

,name,score
0,#DRIVE,69
1,#IDARB,77
2,#KILLALLZOMBIES,53
3,'Splosion Man,84
4,.detuned,46
...,...,...
15765,oOo: Ascension,70
15766,rain,72
15767,theHunter: Call of the Wild,64
15768,uDraw Studio,71


In [9]:
names_list = list(dict.fromkeys(metacritics_df['name'])) #list of 1 instance of game name from metacritics df
scores_avg = []
index = 0
count_same_names = 1
score_sum = 0
i = 0

for j in range(1, len(metacritics_df)):
    score_sum += metacritics_df.score.loc[i]
    if metacritics_df.name.loc[j] == metacritics_df.name.loc[i]:
        count_same_names += 1
    else:
        scores_avg.append(score_sum/count_same_names)
        count_same_names = 1
        score_sum = 0
        if j+1 == len(metacritics_df):
            scores_avg.append(metacritics_df.score.loc[j])
    i+=1

In [10]:
metacritics_df.drop_duplicates(subset='name', inplace=True)
metacritics_df

,name,score
0,#DRIVE,69
1,#IDARB,77
2,#KILLALLZOMBIES,53
3,'Splosion Man,84
4,.detuned,46
...,...,...
15765,oOo: Ascension,70
15766,rain,72
15767,theHunter: Call of the Wild,64
15768,uDraw Studio,71


In [11]:
# some games are dustributed for multiple platforms and each plaform had different rating score
# the new column 'scores_avg' is the average of game scores from different platforms 
metacritics_df['scores_avg'] = scores_avg 
metacritics_df.drop(['score'], axis=1, inplace=True)
metacritics_df

,name,scores_avg
0,#DRIVE,69.0
1,#IDARB,77.0
2,#KILLALLZOMBIES,53.0
3,'Splosion Man,84.0
4,.detuned,46.0
...,...,...
15765,oOo: Ascension,70.0
15766,rain,72.0
15767,theHunter: Call of the Wild,64.0
15768,uDraw Studio,71.0


In [12]:
metacritics_df.reset_index(drop=True, inplace=True)
metacritics_df.to_csv("metacritics_final.csv")

In [13]:
final_df = read_df("final_df.csv")
metacritics_df = read_df("metacritics_final.csv")

In [14]:
final_df['score'] = 0

In [15]:
# filling score values using the metacritics scores dataframe
i=0
for nm in metacritics_df['name']:
    if nm in final_df['name'].values:
        indx = final_df[final_df['name']==nm].index
        final_df.at[indx, 'score'] = metacritics_df.at[i,'scores_avg']
    i+=1

In [16]:
final_df.to_csv("final_df.csv", index=False)

In [17]:
final_df.drop(columns=["Unnamed: 0"], inplace=True)
final_df.reset_index(drop=True)
final_df

,name,developer,publisher,positive,negative,owners,average_playtime,mid_playtime,price,score
0,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5718191,761211,"50,000,000 .. 100,000,000",28846,6493,0,80.666667
1,Dota 2,Valve,Valve,1467658,297030,"100,000,000 .. 200,000,000",38219,964,0,90.000000
2,Grand Theft Auto V,Rockstar North,Rockstar Games,1153983,208800,"20,000,000 .. 50,000,000",12460,6441,1480,96.800000
3,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1146769,892200,"50,000,000 .. 100,000,000",21155,6620,0,0.000000
4,Terraria,Re-Logic,Re-Logic,951689,20646,"20,000,000 .. 50,000,000",6126,1739,499,82.400000
...,...,...,...,...,...,...,...,...,...,...
9618,HEDE Game Engine,Hede Games,Hede Games,0,7,"50,000 .. 100,000",0,0,99,0.000000
9619,League Space,EURO GAMES STUDIO,EURO GAMES STUDIO (ESP),0,1,"50,000 .. 100,000",0,0,1199,0.000000
9620,RagDoll MadDoll,Team Booky,Dystopian Edge Publishing,0,1,"50,000 .. 100,000",0,0,124,0.000000
9621,Operation: VICUS,"Ilja Soutchilin, Tom Berger","Ilja Soutchilin, Tom Berger",0,2,"50,000 .. 100,000",0,0,799,0.000000


In [18]:
# filling scores for games with score = 0
# based on positive & negative values

pos_indx = 3
neg_indx = 4

for indx, row in final_df.iterrows():
    score_val = row.iloc[-1]
    pos_val = row.iloc[pos_indx]
    neg_val = row.iloc[neg_indx]
    if score_val == 0:
        if pos_val == 0:
            new_score = 0
        elif neg_val == 0:
            new_score = pos_val
        else:
            if pos_val > neg_val:
                new_score = 100-((neg_val/pos_val)*100)
            elif pos_val < neg_val:
                new_score = 100-((pos_val/neg_val)*100)
            else:
                new_score = 50
        final_df.at[indx,'score'] = new_score

In [19]:
# calculating avg for owners of a game
owner_indx = 5

for i, row in final_df.iterrows():
    owners = row.iloc[owner_indx]
    
    x = owners.replace(',', '').replace('.', '').split()
    avg = (float(x[0]) + float(x[1]))/2
    final_df.at[i, 'owners'] = avg

In [20]:
final_df.rename(columns={'owners' : 'owners_approx'}, inplace=True)

In [21]:
final_df.to_csv("final_df.csv", index=False)

In [22]:
df = read_df("final_df.csv")
df[(df['score']==0)].count()

name                42
developer           42
publisher           42
positive            42
negative            42
owners_approx       42
average_playtime    42
mid_playtime        42
price               42
score               42
dtype: int64

In [23]:
# dropping rows with 0 score
df.drop(df[df.score == 0].index, inplace=True)

In [24]:
df.reset_index(drop=True, inplace=True)

In [25]:
df.to_csv("final_df.csv", index=False)

In [26]:
df

,name,developer,publisher,positive,negative,owners_approx,average_playtime,mid_playtime,price,score
0,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5718191,761211,75000000.0,28846,6493,0,80.666667
1,Dota 2,Valve,Valve,1467658,297030,150000000.0,38219,964,0,90.000000
2,Grand Theft Auto V,Rockstar North,Rockstar Games,1153983,208800,35000000.0,12460,6441,1480,96.800000
3,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1146769,892200,75000000.0,21155,6620,0,22.198804
4,Terraria,Re-Logic,Re-Logic,951689,20646,35000000.0,6126,1739,499,82.400000
...,...,...,...,...,...,...,...,...,...,...
9576,Sands of Fire,Three-Sixty Pacific,Ziggurat,1,0,350000.0,0,0,349,1.000000
9577,Modern Arena,Indie,Indie,1,1,75000.0,0,0,0,50.000000
9578,Zephyr,New World Computing,Ziggurat,1,0,75000.0,0,0,454,1.000000
9579,Squares,Jason Swisher,Jason Swisher,1,0,150000.0,0,0,99,1.000000


In [27]:
# game defined as successful if score >= 75
is_successful = [1 if i>=75 else 0 for i in df.score]

In [28]:
df['is_successful'] = is_successful

In [29]:
df.to_csv("final_df.csv", index=False)

In [30]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9581 entries, 0 to 9580
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              9581 non-null   object 
 1   developer         9544 non-null   object 
 2   publisher         9556 non-null   object 
 3   positive          9581 non-null   int64  
 4   negative          9581 non-null   int64  
 5   owners_approx     9581 non-null   float64
 6   average_playtime  9581 non-null   int64  
 7   mid_playtime      9581 non-null   int64  
 8   price             9581 non-null   int64  
 9   score             9581 non-null   float64
 10  is_successful     9581 non-null   int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 823.5+ KB


In [31]:
for indx, row in df.iterrows():
    dev_name = row.iloc[1]
    pub_name = row.iloc[2]
    
    if dev_name is np.nan:
        df.at[indx, 'developer'] = pub_name
    if pub_name is np.nan:
        df.at[indx, 'publisher'] = dev_name

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9581 entries, 0 to 9580
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              9581 non-null   object 
 1   developer         9580 non-null   object 
 2   publisher         9580 non-null   object 
 3   positive          9581 non-null   int64  
 4   negative          9581 non-null   int64  
 5   owners_approx     9581 non-null   float64
 6   average_playtime  9581 non-null   int64  
 7   mid_playtime      9581 non-null   int64  
 8   price             9581 non-null   int64  
 9   score             9581 non-null   float64
 10  is_successful     9581 non-null   int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 823.5+ KB


In [33]:
df.to_csv("final_df.csv", index=False)

In [34]:
df.describe(include='all')

,name,developer,publisher,positive,negative,owners_approx,average_playtime,mid_playtime,price,score,is_successful
count,9581,9580,9580,9.581000e+03,9581.000000,9.581000e+03,9581.000000,9581.000000,9581.000000,9581.000000,9581.000000
unique,9581,6427,4849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Counter-Strike: Global Offensive,Valve,Ubisoft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,32,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,7.533910e+03,1213.027450,6.360933e+05,735.984135,613.999165,494.337647,67.235737,0.478134
std,NaN,NaN,NaN,7.114004e+04,13405.760653,2.984572e+06,6584.627619,7123.876017,730.726438,23.946697,0.499548
min,NaN,NaN,NaN,0.000000e+00,0.000000,3.500000e+04,0.000000,0.000000,0.000000,0.124844,0.000000
25%,NaN,NaN,NaN,2.100000e+02,62.000000,7.500000e+04,7.000000,8.000000,49.000000,56.000000,0.000000
50%,NaN,NaN,NaN,7.510000e+02,166.000000,1.500000e+05,168.000000,155.000000,239.000000,73.815968,0.000000
75%,NaN,NaN,NaN,2.671000e+03,490.000000,3.500000e+05,420.000000,362.000000,674.000000,84.806507,1.000000


In [35]:
# int to float, cents to dolars
df['price'] = df['price'].astype(float)
for indx, row in df.iterrows():
    df.at[indx, 'price'] = row.iloc[8]/float(100)
df

,name,developer,publisher,positive,negative,owners_approx,average_playtime,mid_playtime,price,score,is_successful
0,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,5718191,761211,75000000.0,28846,6493,0.00,80.666667,1
1,Dota 2,Valve,Valve,1467658,297030,150000000.0,38219,964,0.00,90.000000,1
2,Grand Theft Auto V,Rockstar North,Rockstar Games,1153983,208800,35000000.0,12460,6441,14.80,96.800000,1
3,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",1146769,892200,75000000.0,21155,6620,0.00,22.198804,0
4,Terraria,Re-Logic,Re-Logic,951689,20646,35000000.0,6126,1739,4.99,82.400000,1
...,...,...,...,...,...,...,...,...,...,...,...
9576,Sands of Fire,Three-Sixty Pacific,Ziggurat,1,0,350000.0,0,0,3.49,1.000000,0
9577,Modern Arena,Indie,Indie,1,1,75000.0,0,0,0.00,50.000000,0
9578,Zephyr,New World Computing,Ziggurat,1,0,75000.0,0,0,4.54,1.000000,0
9579,Squares,Jason Swisher,Jason Swisher,1,0,150000.0,0,0,0.99,1.000000,0


In [36]:
df.to_csv("final_df.csv", index=False)